<a href="https://colab.research.google.com/github/Aadithyakris/PriceTracker/blob/main/hetero_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#cell 0
# Install the core library only
!pip install -q torch-geometric

# Check if we can run basic HeteroData without the compiled helpers
import torch
from torch_geometric.data import HeteroData
try:
    data = HeteroData()
    print(" Core PyTorch Geometric is working without the heavy scatter/sparse binaries!")
except Exception as e:
    print(f" Error: {e}")

 Core PyTorch Geometric is working without the heavy scatter/sparse binaries!


In [ ]:
#cell 1
import os
import spacy
import pickle
import pandas as pd
from tqdm import tqdm
from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2.Paths
DATASET_PATH = '/content/drive/MyDrive/authorship_corpus_400'
SAVE_PATH = '/content/drive/MyDrive/authorship_attribution/processed_data_2'

# Create save directory if it doesn't exist
os.makedirs(SAVE_PATH, exist_ok=True)

# 3. Load spaCy model (Small model is faster for Colab Free Tier)
try:
    nlp = spacy.load("en_core_web_sm")
except:
    !python -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")

print(" Setup Complete. Drive mounted and spaCy loaded.")

Mounted at /content/drive
 Setup Complete. Drive mounted and spaCy loaded.


In [ ]:
def extract_linguistic_features(text, doc_id, author_label):
    doc = nlp(text)
    features = {
        'doc_id': doc_id,
        'author': author_label,
        'tokens': [],
        'lemmas': [], # LAYER A
        'pos': [],    # LAYER B
        'dep_edges': [], # LAYER B
        'entities': []   # LAYER C
    }

    for token in doc:
        features['tokens'].append(token.text.lower())
        features['lemmas'].append(token.lemma_.lower())
        features['pos'].append(token.pos_)
        features['dep_edges'].append((token.head.i, token.dep_, token.i))

    for ent in doc.ents:
        features['entities'].append({
            'label': ent.label_, # Using Type instead of Text to avoid content-bias
            'indices': list(range(ent.start, ent.end))
        })
    return features

In [ ]:
#cell 3
def process_split(split_name):
    output_file = os.path.join(SAVE_PATH, f"{split_name}_features.pkl")

    # --- ADDED: Check if file already exists ---
    if os.path.exists(output_file):
        print(f" {split_name}_features.pkl already exists in Drive. Skipping processing.")
        return f"Loaded from Drive: {output_file}"

    split_path = os.path.join(DATASET_PATH, split_name)
    all_processed_data = []

    authors = [d for d in os.listdir(split_path) if os.path.isdir(os.path.join(split_path, d))]

    print(f" Processing {split_name} split...")
    for author in authors:
        author_path = os.path.join(split_path, author)
        files = [f for f in os.listdir(author_path) if f.endswith('.txt')]

        for file_name in tqdm(files, desc=f"Author: {author}"):
            file_path = os.path.join(author_path, file_name)
            try:
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    text = f.read()

                # Truncate for RAM safety
                if len(text) > 50000: text = text[:50000]

                features = extract_linguistic_features(text, file_name, author)
                all_processed_data.append(features)
            except Exception as e:
                print(f" Error processing {file_name}: {e}")

    with open(output_file, 'wb') as f:
        pickle.dump(all_processed_data, f)

    return f"Done: {len(all_processed_data)} files processed for {split_name}."

# --- EXECUTION ---
print(process_split('train'))
print(process_split('test'))

 Processing train split...


Author: Arthur_Conan_Doyle: 100%|██████████| 32/32 [01:01<00:00,  1.94s/it]


Done: 314 files processed for train.
 Processing test split...


Author: Arthur_Conan_Doyle: 100%|██████████| 8/8 [00:15<00:00,  1.93s/it]


Done: 80 files processed for test.


In [ ]:
#cell 4
def check_processed_data(split_name):
    file_path = os.path.join(SAVE_PATH, f"{split_name}_features.pkl")

    if not os.path.exists(file_path):
        print(f" Error: {file_path} not found. Did you run the processing loop?")
        return

    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    print(f" Validation for {split_name}:")
    print(f"Total documents parsed: {len(data)}")

    # Check a sample document
    sample = data[0]
    print(f"Sample Author: {sample['author']}")
    print(f"Tokens extracted (first 5): {sample['tokens'][:5]}")
    print(f"POS tags extracted (first 5): {sample['pos'][:5]}")
    print(f"Dependency relations count: {len(sample['dep_edges'])}")
    print(f"Entities found: {len(sample['entities'])}")

# Run the check
check_processed_data('train')

 Validation for train:
Total documents parsed: 314
Sample Author: Thomas_Hardy
Tokens extracted (first 5): ['stories', 'by', 'english', 'authors', '\n\n']
POS tags extracted (first 5): ['NOUN', 'ADP', 'PROPN', 'PROPN', 'SPACE']
Dependency relations count: 12189
Entities found: 396


In [7]:
#cell 5
import os
import pickle

# 1. Load the raw features we saved in Cell 3
features_path = os.path.join(SAVE_PATH, "train_features.pkl")

with open(features_path, 'rb') as f:
    train_data = pickle.load(f)

def create_ontology_mappings(all_data):
    """Calculates the global vocabulary for all ontology layers."""
    # LAYER B: Syntactic types (Noun, Verb, etc.)
    pos_types = sorted(list(set(p for doc in all_data for p in doc['pos'])))

    # LAYER C: Semantic types (PERSON, GPE, DATE, etc.)
    ent_types = sorted(list(set(e['label'] for doc in all_data for e in doc['entities'])))

    # LAYER E: Author labels
    authors = sorted(list(set(doc['author'] for doc in all_data)))

    return {
        'pos': {t: i for i, t in enumerate(pos_types)},
        'ent': {t: i for i, t in enumerate(ent_types)},
        'author': {t: i for i, t in enumerate(authors)}
    }

# 2. Generate the maps used by the Graph Builder and the Model
ontology_maps = create_ontology_mappings(train_data)

# Extract individual maps for easy access in later cells
pos_map = ontology_maps['pos']
ent_map = ontology_maps['ent']
author_map = ontology_maps['author']

print(" Mappings Created Successfully!")
print(f" POS nodes: {len(pos_map)} | Entity types: {len(ent_map)} | Authors: {len(author_map)}")

 Mappings Created Successfully!
 POS nodes: 18 | Entity types: 18 | Authors: 10


In [8]:
#cell 6
import torch
from torch_geometric.data import HeteroData

def build_ontology_graph(doc_features, ontology_maps):
    data = HeteroData()
    pos_map = ontology_maps['pos']
    ent_map = ontology_maps['ent']
    author_map = ontology_maps['author']

    # --- 1. Nodes ---
    num_tokens = len(doc_features['tokens'])
    data['token'].node_id = torch.arange(num_tokens)

    # Initialize POS and Entity types as nodes with one-hot features
    data['pos'].x = torch.eye(len(pos_map))
    data['ent_type'].x = torch.eye(len(ent_map))

    # LAYER A: Lemma Nodes (Lexical root)
    unique_lemmas = list(set(doc_features['lemmas']))
    lemma_to_idx = {l: i for i, l in enumerate(unique_lemmas)}
    data['lemma'].num_nodes = len(unique_lemmas)

    # --- 2. Relations (Ontology Edges) ---

    # LAYER A: token -> has_lemma -> lemma
    t_idx, l_idx = [], []
    for i, lemma in enumerate(doc_features['lemmas']):
        t_idx.append(i); l_idx.append(lemma_to_idx[lemma])
    data['token', 'has_lemma', 'lemma'].edge_index = torch.tensor([t_idx, l_idx], dtype=torch.long)

    # LAYER B: token -> has_pos -> pos (Syntactic)
    p_indices = [pos_map[p] for p in doc_features['pos']]
    data['token', 'has_pos', 'pos'].edge_index = torch.tensor([list(range(num_tokens)), p_indices], dtype=torch.long)

    # LAYER B: token -> depends_on -> token (Syntactic Structure)
    heads = [e[0] for e in doc_features['dep_edges']]
    children = [e[2] for e in doc_features['dep_edges']]
    data['token', 'depends_on', 'token'].edge_index = torch.tensor([heads, children], dtype=torch.long)

    # LAYER C: token -> is_entity -> ent_type (Semantic)
    t_ent_idx, type_idx = [], []
    for ent in doc_features['entities']:
        # FIX: Use 'indices' which matches your Cell 2 output
        for i in ent['indices']:
            if i < num_tokens:
                t_ent_idx.append(i)
                type_idx.append(ent_map[ent['label']])

    if t_ent_idx:
        data['token', 'is_entity', 'ent_type'].edge_index = torch.tensor([t_ent_idx, type_idx], dtype=torch.long)

    # LAYER E: Author Label
    data['author_id'] = author_map[doc_features['author']]

    return data

In [ ]:
#cell 7
def convert_and_save_ontology_graphs(split_name):
    with open(os.path.join(SAVE_PATH, f"{split_name}_features.pkl"), 'rb') as f:
        features_list = pickle.load(f)

    graph_list = []
    print(f" Building Ontology Graphs for {split_name}...")

    for feat in tqdm(features_list, desc=f"Graphing {split_name}"):
        # Pass the ontology_maps dictionary we built in Cell 5
        graph = build_ontology_graph(feat, ontology_maps)
        graph_list.append(graph)

    output_path = os.path.join(SAVE_PATH, f"{split_name}_graphs.pt")
    torch.save(graph_list, output_path)
    print(f" Saved {len(graph_list)} multi-layered graphs to {output_path}")

# Run conversion
convert_and_save_ontology_graphs('train')
convert_and_save_ontology_graphs('test')

 Building Ontology Graphs for train...


Graphing train: 100%|██████████| 314/314 [00:04<00:00, 76.57it/s] 


 Saved 314 multi-layered graphs to /content/drive/MyDrive/authorship_attribution/processed_data_2/train_graphs.pt
 Building Ontology Graphs for test...


Graphing test: 100%|██████████| 80/80 [00:00<00:00, 115.10it/s]


 Saved 80 multi-layered graphs to /content/drive/MyDrive/authorship_attribution/processed_data_2/test_graphs.pt


In [ ]:
#cell 8
import torch_geometric

# Allow the specific PyG classes that the unpickler is complaining about
torch.serialization.add_safe_globals([
    torch_geometric.data.storage.BaseStorage,
    torch_geometric.data.storage.NodeStorage,
    torch_geometric.data.storage.EdgeStorage,
    torch_geometric.data.storage.GlobalStorage,
    torch_geometric.data.HeteroData
])

def check_graph_health(path):
    # Try loading with weights_only=False since these are complex objects, not just weights
    graphs = torch.load(path, weights_only=False)
    sample = graphs[0]

    print(f" Graph Health Check for {os.path.basename(path)}:")
    print(f"Node Types: {sample.node_types}")
    print(f"Edge Types: {sample.edge_types}")

    # Check connectivity
    # Using 'token', 'has_pos', 'pos' as the keys
    num_pos_edges = sample['token', 'has_pos', 'pos'].edge_index.shape[1]
    num_dep_edges = sample['token', 'depends_on', 'token'].edge_index.shape[1]

    print(f"Token-POS connections: {num_pos_edges}")
    print(f"Dependency connections: {num_dep_edges}")

    if num_pos_edges == 0 or num_dep_edges == 0:
        print(" Warning: Empty relations detected!")
    else:
        print(" Graph is structurally sound and ready for Features.")

check_graph_health(os.path.join(SAVE_PATH, "train_graphs.pt"))

 Graph Health Check for train_graphs.pt:
Node Types: ['token', 'pos', 'ent_type', 'lemma']
Edge Types: [('token', 'has_lemma', 'lemma'), ('token', 'has_pos', 'pos'), ('token', 'depends_on', 'token'), ('token', 'is_entity', 'ent_type')]
Token-POS connections: 12189
Dependency connections: 12189
 Graph is structurally sound and ready for Features.


In [ ]:
#cell 9
from transformers import AutoTokenizer, AutoModel
import torch

# Use DistilBERT for Colab Free Tier efficiency
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModel.from_pretrained(model_name).to('cuda' if torch.cuda.is_available() else 'cpu')

def get_bert_embeddings(text_list):
    # Process in small chunks to avoid memory errors
    inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=512).to(bert_model.device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    # Use the last hidden state
    return outputs.last_hidden_state.mean(dim=1) # Shape: [num_tokens, 768]

print(" DistilBERT Loaded and ready.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

 DistilBERT Loaded and ready.


In [ ]:
#cell 10
def generate_ontology_shards(split_name):
    shard_dir = os.path.join(SAVE_PATH, f"{split_name}_shards")
    os.makedirs(shard_dir, exist_ok=True)

    features_path = os.path.join(SAVE_PATH, f"{split_name}_features.pkl")
    with open(features_path, 'rb') as f:
        raw_features = pickle.load(f)

    print(f" Generating Ontology Shards for {split_name}...")

    bert_model.eval()
    with torch.no_grad():
        for i, feat in enumerate(tqdm(raw_features)):
            # 1. Build the Graph structure
            graph = build_ontology_graph(feat, ontology_maps)

            # 2. Add BERT embeddings to the 'token' nodes
            doc_text = " ".join(feat['tokens'][:512])
            inputs = tokenizer(doc_text, return_tensors="pt", padding=True, truncation=True, max_length=512).to('cuda')
            outputs = bert_model(**inputs)

            # Match embedding length to token length (distilbert has [CLS] and [SEP])
            emb = outputs.last_hidden_state[0][1:-1, :] # Remove special tokens
            num_graph_tokens = graph['token'].node_id.size(0)

            # Align lengths if spaCy and BERT tokenization differ slightly
            if emb.size(0) > num_graph_tokens:
                graph['token'].x = emb[:num_graph_tokens, :].cpu()
            else:
                padding = torch.zeros((num_graph_tokens - emb.size(0), 768))
                graph['token'].x = torch.cat([emb.cpu(), padding], dim=0)

            # 3. Save as individual shard
            torch.save(graph, os.path.join(shard_dir, f"graph_{i}.pt"))

    print(f"✅ Created shards in: {shard_dir}")

# EXECUTE
generate_ontology_shards('train')
generate_ontology_shards('test')

🛠️ Generating Ontology Shards for train...


100%|██████████| 314/314 [04:50<00:00,  1.08it/s]


✅ Created shards in: /content/drive/MyDrive/authorship_attribution/processed_data_2/train_shards
🛠️ Generating Ontology Shards for test...


100%|██████████| 80/80 [01:15<00:00,  1.06it/s]

✅ Created shards in: /content/drive/MyDrive/authorship_attribution/processed_data_2/test_shards


In [3]:
#cell 11
# Standard Library Imports
import os
import gc
import pickle
import random
import spacy
import pandas as pd
from google.colab import drive

# 2. PyTorch & Geometric Imports (Done in one block to avoid registration errors)
import torch
import torch.nn as nn
from tqdm import tqdm

# IMPORTANT: Import geometric components safely
import torch_geometric
from torch_geometric.nn import HGTConv, global_mean_pool
from torch_geometric.data import HeteroData

# 3. Path & Logic Setup
drive.mount('/content/drive')
DATASET_PATH = '/content/drive/MyDrive/dataset'
SAVE_PATH = '/content/drive/MyDrive/authorship_attribution/processed_data_2'

# Allowlist for PyTorch 2.6+ security
torch.serialization.add_safe_globals([
    torch_geometric.data.storage.BaseStorage,
    torch_geometric.data.storage.NodeStorage,
    torch_geometric.data.storage.EdgeStorage,
    torch_geometric.data.storage.GlobalStorage,
    torch_geometric.data.HeteroData,
    torch._tensor._rebuild_from_type_v2 # Common fix for PyTorch 2.9
])

# 4. Reload Metadata
features_path = os.path.join(SAVE_PATH, "train_features.pkl")
with open(features_path, 'rb') as f:
    train_data = pickle.load(f)

pos_types = sorted(list(set(p for doc in train_data for p in doc['pos'])))
authors = sorted(list(set(doc['author'] for doc in train_data)))
pos_map = {t: i for i, t in enumerate(pos_types)}
author_map = {t: i for i, t in enumerate(authors)}

print(f" Environment Restored without RuntimeError.")
print(f"Ready to process {len(author_map)} authors.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 Environment Restored without RuntimeError.
Ready to process 10 authors.


In [4]:
#cell 12
import os
import torch
from tqdm import tqdm

def stream_augment_shards(split_name):
    # Path to the shards already existing on your Drive
    drive_shard_dir = os.path.join(SAVE_PATH, f"{split_name}_shards")
    # New local destination on SSD
    local_dir = f"/content/{split_name}_shards"
    os.makedirs(local_dir, exist_ok=True)

    if not os.path.exists(drive_shard_dir):
        print(f" Error: Could not find shards at {drive_shard_dir}. Please verify the path.")
        return

    shard_files = sorted([f for f in os.listdir(drive_shard_dir) if f.endswith('.pt')])
    print(f" Augmenting {split_name} by streaming from Drive to Local Disk...")

    new_count = 0
    for f in tqdm(shard_files):
        # Load ONLY ONE graph at a time (RAM remains low)
        # graph = torch.load(os.path.join(drive_shard_dir, f), weights_only=False)
        graph = torch.load(os.path.join(drive_shard_dir, f), map_location='cpu', weights_only=False)

        # 1. Save Original
        torch.save(graph, os.path.join(local_dir, f"graph_{new_count}.pt"))
        new_count += 1

        # 2. Augment: Only if the graph is large enough
        # We take a style fragment (simulating a shorter excerpt)
        if graph['token'].node_id.size(0) > 200:
            torch.save(graph, os.path.join(local_dir, f"graph_{new_count}.pt"))
            new_count += 1

        # Clear the single graph from memory immediately
        del graph

    print(f" Created {new_count} local shards at {local_dir}.")

# Execute
stream_augment_shards('train')
stream_augment_shards('test')

 Augmenting train by streaming from Drive to Local Disk...


100%|██████████| 314/314 [08:19<00:00,  1.59s/it]


 Created 628 local shards at /content/train_shards.
 Augmenting test by streaming from Drive to Local Disk...


100%|██████████| 80/80 [02:06<00:00,  1.58s/it]

 Created 160 local shards at /content/test_shards.


In [5]:
# Run if  error about metadata
sample = torch.load('/content/train_shards/graph_0.pt', weights_only=False)
metadata = sample.metadata()

In [17]:
#cell 13
import torch.nn as nn
from torch_geometric.nn import HGTConv, global_mean_pool

class AuthorOntologyHGT(nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers, metadata, ontology_maps):
        super().__init__()
        self.node_types = metadata[0]
        self.hidden_channels = hidden_channels

        self.lin_dict = nn.ModuleDict()
        for node_type in self.node_types:
            # Set input dimensions based on the ontology layer
            if node_type == 'token':
                in_dim = 768  # DistilBERT
            elif node_type == 'pos':
                in_dim = len(ontology_maps['pos'])
            elif node_type == 'ent_type':
                in_dim = len(ontology_maps['ent'])
            else:
                # Lemma nodes: we give them a 256-dim space to start with
                in_dim = hidden_channels

            self.lin_dict[node_type] = nn.Linear(in_dim, hidden_channels)

        self.convs = nn.ModuleList()
        for _ in range(num_layers):
            self.convs.append(HGTConv(hidden_channels, hidden_channels, metadata, num_heads))

        self.classifier = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_channels, out_channels)
        )

    def forward(self, x_dict, edge_index_dict, batch_dict_token):
        x_out = {}
        for node_type in self.node_types:
            if node_type in x_dict:
                x = x_dict[node_type]
            else:
                # FIX: Create features with size [num_nodes, 256] instead of [num_nodes, 1]
                # We determine the number of nodes from the edge index or metadata
                if node_type == 'lemma':
                    # Find how many unique lemmas are in this specific shard
                    # It's the max index found in the 'has_lemma' relations
                    num_nodes = edge_index_dict[('token', 'has_lemma', 'lemma')][1].max().item() + 1
                else:
                    num_nodes = 1 # Fallback

                x = torch.zeros((num_nodes, self.hidden_channels)).to(batch_dict_token.device)

            x_out[node_type] = self.lin_dict[node_type](x).relu()

        for conv in self.convs:
            x_out = conv(x_out, edge_index_dict)

        return self.classifier(global_mean_pool(x_out['token'], batch_dict_token))

# class AuthorOntologyHGT(nn.Module):
#     def __init__(self, hidden_channels, out_channels, num_heads, num_layers, metadata, ontology_maps):
#         super().__init__()
#         self.node_types = metadata[0]
#         self.lin_dict = nn.ModuleDict()

#         for node_type in self.node_types:
#             # Set input dimensions based on the ontology layer
#             if node_type == 'token':
#                 in_dim = 768  # DistilBERT
#             elif node_type == 'pos':
#                 in_dim = len(ontology_maps['pos'])
#             elif node_type == 'ent_type':
#                 in_dim = len(ontology_maps['ent'])
#             elif node_type == 'lemma':
#                 # Lemmas don't have features, we'll use a constant and learn its projection
#                 in_dim = hidden_channels

#             self.lin_dict[node_type] = nn.Linear(in_dim, hidden_channels)

#         self.convs = nn.ModuleList()
#         for _ in range(num_layers):
#             self.convs.append(HGTConv(hidden_channels, hidden_channels, metadata, num_heads))

#         self.classifier = nn.Sequential(
#             nn.Linear(hidden_channels, hidden_channels),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(hidden_channels, out_channels)
#         )

#     def forward(self, x_dict, edge_index_dict, batch_dict_token):
#         # 1. Initial Projections
#         for node_type, x in x_dict.items():
#             # Handle nodes without pre-computed features (like lemma)
#             if node_type == 'lemma':
#                 # Create a temporary zero feature for lemmas to be updated via message passing
#                 x = torch.zeros((x_dict['token'].size(0), HIDDEN_CHANNELS)).to(x_dict['token'].device)

#             x_dict[node_type] = self.lin_dict[node_type](x).relu()

#         # 2. Ontology Message Passing
#         # Information flows between Token -> POS, Token -> Lemma, Token -> Entity Type
#         for conv in self.convs:
#             x_dict = conv(x_dict, edge_index_dict)

#         # 3. Global Pooling (Discourse Layer)
#         out = global_mean_pool(x_dict['token'], batch_dict_token)
#         return self.classifier(out)

# --- RE-INITIALIZATION ---
HIDDEN_CHANNELS = 256
NUM_HEADS = 8
NUM_LAYERS = 3

# Load a single shard to extract the new ontology metadata
sample = torch.load("/content/train_shards/graph_0.pt", map_location='cpu', weights_only=False)
metadata = sample.metadata()

model = AuthorOntologyHGT(HIDDEN_CHANNELS, len(author_map), NUM_HEADS, NUM_LAYERS, metadata, ontology_maps)
print(" Ontology-Aware HGT Model Initialized.")

 Ontology-Aware HGT Model Initialized.


In [20]:
# 1. Initialize with 256 channels
HIDDEN_CHANNELS = 256

# 2. Re-create model
model = AuthorOntologyHGT(
    hidden_channels=HIDDEN_CHANNELS,
    out_channels=len(author_map),
    num_heads=8,
    num_layers=3,
    metadata=metadata,
    ontology_maps=ontology_maps
)

# 3. Run the training
train_ontology_model(model)

Epoch 1 [Ontology Training]: 100%|██████████| 628/628 [04:38<00:00,  2.26it/s, loss=2.2725]


📊 Epoch 1 | Avg Loss: 2.3083 | Acc: 0.1000
 New Best Ontology Model Saved!


Epoch 2 [Ontology Training]: 100%|██████████| 628/628 [04:38<00:00,  2.26it/s, loss=2.2698]


📊 Epoch 2 | Avg Loss: 2.3029 | Acc: 0.1000


Epoch 3 [Ontology Training]: 100%|██████████| 628/628 [04:45<00:00,  2.20it/s, loss=2.2711]


...

📊 Epoch 28 | Avg Loss: 0.6892 | Acc: 0.1375


Epoch 29 [Ontology Training]: 100%|██████████| 628/628 [04:39<00:00,  2.24it/s, loss=0.0000]


📊 Epoch 29 | Avg Loss: 0.6904 | Acc: 0.1250


Epoch 30 [Ontology Training]: 100%|██████████| 628/628 [04:39<00:00,  2.25it/s, loss=2.6046]


📊 Epoch 30 | Avg Loss: 0.6954 | Acc: 0.1500


In [19]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

def train_ontology_model(model, epochs=30):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=0.0003, weight_decay=0.05)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
    criterion = torch.nn.CrossEntropyLoss()

    train_dir, test_dir = "/content/train_shards", "/content/test_shards"
    train_files = [f for f in os.listdir(train_dir) if f.endswith('.pt')]
    test_files = [f for f in os.listdir(test_dir) if f.endswith('.pt')]

    best_acc = 0

    for epoch in range(epochs):
        model.train()
        random.shuffle(train_files)
        total_loss, count = 0, 0

        pbar = tqdm(train_files, desc=f"Epoch {epoch+1} [Ontology Training]")
        for f in pbar:
            try:
                data = torch.load(os.path.join(train_dir, f), map_location=device, weights_only=False)
                optimizer.zero_grad()

                batch_idx = torch.zeros(data['token'].x.size(0), dtype=torch.long).to(device)
                out = model(data.x_dict, data.edge_index_dict, batch_idx)

                label = torch.tensor([data.author_id]).to(device)
                loss = criterion(out, label)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                count += 1
                pbar.set_postfix({'loss': f"{loss.item():.4f}"})

                del data
                if count % 10 == 0: torch.cuda.empty_cache()
            except Exception as e:
                continue # Skip corrupted shards

        # Evaluation
        model.eval()
        correct, total_test = 0, 0
        with torch.no_grad():
            for f in test_files:
                try:
                    data = torch.load(os.path.join(test_dir, f), map_location=device, weights_only=False)
                    batch_idx = torch.zeros(data['token'].x.size(0), dtype=torch.long).to(device)
                    out = model(data.x_dict, data.edge_index_dict, batch_idx)
                    if out.argmax(dim=-1).item() == data.author_id:
                        correct += 1
                    total_test += 1
                    del data
                except:
                    continue

        acc = correct / total_test if total_test > 0 else 0
        scheduler.step(acc)

        if count > 0:
            print(f"📊 Epoch {epoch+1} | Avg Loss: {total_loss/count:.4f} | Acc: {acc:.4f}")
        else:
            print(f"📊 Epoch {epoch+1} | No successful training steps. | Acc: {acc:.4f}")

        if acc > best_acc:
            best_acc = acc
            torch.save(model.state_dict(), os.path.join(SAVE_PATH, "ontology_hgt_best.pth"))
            print(" New Best Ontology Model Saved!")

# START
train_ontology_model(model)

Starting Training...
Testing file: graph_345.pt


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2001x1 and 256x256)

In [ ]:
torch.cuda.empty_cache()